#### 从压缩包提取数据，获得指定symbol的交易数据并导出为csv
来源：https://zhuanlan.zhihu.com/p/51418002 <br/>
学习要点：<br/>
（1）tarfile包的运用，如何从压缩包中提取指定文件名的文件句柄；<br/>
（2）reduce函数，从一个list里连续地将元素作为输入调用函数（可以给定初值），感觉也可以用for循环完成；<br/>
（3）<br/>
list操作：.append()；<br/>
pandas操作：pd.DataFrame()生成一个空的df；.drop_duplicates([var])去重；.to_csv(path, columns=[])输出到csv文件

In [ ]:
"""
Usage: parse_data.py --company=<company>
"""
import os
import tarfile
import pandas as pd
from pandas import errors as pd_errors
from functools import reduce
from docopt import docopt
 
args = docopt(doc=__doc__, argv=None,
help=True, version=None,
options_first=False)
 
years = [2016]
company = args['--company']

 
# 获取数据文件列表
data_files_list = []
for year in years:
    year_directory = f'data/{year}'
for file in os.listdir(year_directory):
    if file[-7:] == '.tar.gz':  # 只看满足该后缀的压缩文件夹
        data_files_list.append(f'{year_directory}/{file}')
 

# parse_data，从一个tar压缩包里的"prices.csv"文件中获取指定股票的交易数据
def parse_data(file_name, company_symbol):
    """
    Returns data for the corresponding company
     
    :param file_name: name of the tar file
    :param company_symbol: company symbol
    :type file_name: str
    :type company_symbol: str
    :return: dataframe for the corresponding company data
    :rtype: pd.DataFrame
    """
    tar = tarfile.open(file_name)  # 打开一个tar压缩包，然后tar.extractfile()获取具体的文件句柄
    try:
        price_report = pd.read_csv(tar.extractfile('prices.csv'))
        company_price_data = price_report[price_report['symbol'] == company_symbol]
        return company_price_data
    except (KeyError, pd_errors.EmptyDataError):
        return pd.DataFrame()
     
     
# 获得给定公司的完整数据
company_data = reduce(lambda df, file_name: df.append(parse_data(file_name, company)),
data_files_list,
pd.DataFrame())
# reduce(func, sequeue, initial)函数，逐一将sequeue里的value作为输入调用func，初始为initial
company_data = company_data.sort_values(by=['date'])  #按日期排序
company_data = company_data.drop_duplicates(['symbol', 'date'])  # 去重
 
# 若不存在，则为公司数据创建文件夹
if not os.path.exists('data/company_data'):
    os.makedirs('data/company_data')
 
# 将数据写到CSV 文件中
company_data.to_csv('data/company_data/{company}.csv'.format(company=company),
columns=['date', 'open', 'high', 'low', 'close', 'volume', 'adj_close'],
index=False)